In [1]:
import os
import platform
import textwrap
from enum import Enum
from multiprocessing import Pool

import pandas as pd
from openai import OpenAI
from pydantic import BaseModel, Field
from tqdm.auto import tqdm

MODEL = "gpt-4o-mini-2024-07-18"

client = OpenAI()

if "macOS" in platform.platform():
    # Revert mp's start method to fork on MacOS
    import multiprocessing

    multiprocessing.set_start_method("fork", force=False)

# Parse and Categorize Student Questions

In [38]:
class Category(Enum):
    GOVERNMENT = "government"
    MARKET_ANALYSIS = "market_analysis"
    PHILOSOPHY = "philosophy"
    PUZZLE = "puzzle"
    STRATEGY = "strategy"


class QuestionSummary(BaseModel):
    category: Category = Field(description="The high-level category of the question")
    name: str = Field(description="A descriptive name based on the provided question. Must be three or fewer words")


def work(inputs, client=client):
    _, row = inputs
    prompt = row["prompt"].strip()
    try:
        cot, final_output = row["response"].strip("<think>").split("</think>")
    except ValueError:
        # Some students didn't copy the entire output including the CoT
        cot, final_output = None, row["response"]
    result = {"prompt": prompt, "cot": cot, "final_output": final_output}

    try:
        completion = client.beta.chat.completions.parse(
            model=MODEL,
            messages=[
                {"role": "system", "content": "Extract a summary of the user's question."},
                {"role": "user", "content": prompt},
            ],
            response_format=QuestionSummary,
        )
        result["parse_result"] = completion.choices[0].message.parsed
    except Exception as error:
        result["error"] = {"type": str(type(error)), "msg": str(error)}

    return result


r1_qas = pd.read_csv("r1_students_queries.csv")
with Pool() as p:
    parsed_prompts = [r for r in tqdm(p.imap_unordered(work, r1_qas.iterrows()), total=len(r1_qas))]

# Flatten into a table
recs = []
for r in parsed_prompts:
    rec = r.copy()
    parse = rec.pop("parse_result")
    rec.update({"name": parse.name, "category": parse.category.value})
    recs.append(rec)
df = pd.DataFrame(recs).fillna("<|MISSING_STUDENT_INPUT/OUTPUT|>")

  0%|          | 0/40 [00:00<?, ?it/s]

# Write Out Q&As to Markdown Files

In [89]:
TEMPLATE_QA = """
# {name}
## Prompt
```
{prompt}
```

## Response
### Chain-of-Though/Reasoning Chain (<think>{{cot}}</think>)
```
{cot}
```

### Final Output
```
{final_output}
```
"""

for _, row in df.iterrows():
    name = row["name"]
    filename = os.path.join("..", "r1-responses", f"{name.replace(' ', '_').lower()}.md")
    for_md = {"name": name}
    for_md.update(
        {
            k: "\n".join(textwrap.wrap(row[k], 80, replace_whitespace=False)).strip()
            for k in ["prompt", "cot", "final_output"]
        }
    )
    md = TEMPLATE_QA.format(**for_md)

    with open(filename, "w") as out_file:
        out_file.write(md)

# Write Main MD File

In [90]:
main_md = [
    """
# Musings from R1
Students from BUSN30135 used R1 to answer a variety of questions ranging from valuing a rare violin to assessing the market for teleportation to logic puzzles. This repo contains a handful of these questions, R1's chain of thought when answering, and R1's final response.

## Index of Q&As
""".strip()
]

for category, df_cat in df.sort_values(["category", "name"]).groupby("category"):
    main_md.append(f"### {category.replace('_', ' ').title()}")
    for _, row in df_cat.iterrows():
        name = row["name"]
        fn = f"{name.replace(' ', '_').lower()}.md"
        main_md.append(f"- [{row['name']}]({os.path.join('r1-responses', fn)})")
    main_md.append("")

with open("../README.md", "w") as out_file:
    out_file.write("\n".join(main_md))